# **PROJECT: Coevolutionary Systems**

### **IMPORTS AND INITIALIZATIONS**

In [25]:
import random, numpy
from deap import algorithms, base, creator, tools

#Initializations
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

NUM_SPECIES = 2 #Number of Species
SPECIES_SIZE = 100 #Population of each species 
IND_SIZE = 50 #Individual Size or Bit size
TARGET_SIZE = 100
TARGET_TYPE = 2

toolbox = base.Toolbox()

c:\Users\Marcus\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Marcus\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### **DEFINING FUNCTIONS**

In [26]:
def nicheSchematas(type, size):
    """Produce the desired schemata based on the type required, 2 for half
    length, 4 for quarter length and 8 for eight length.
    """
    rept = int(size/type)
    return ["#" * (i*rept) + "1" * rept + "#" * ((type-i-1)*rept) for i in range(type)]

def initTargetSet(schemata, size):
    """Initialize a target set with noisy string to match based on the
    schematas provided.
    """
    test_set = []
    for _ in range(size):
        test = list(random.randint(0, 1) for _ in range(len(schemata)))
        for i, x in enumerate(schemata):
            if x == "0":
                test[i] = 0
            elif x == "1":
                test[i] = 1
        test_set.append(test)
    return test_set

def matchStrength(x, y):
    """Compute the match strength for the individual *x* on the string *y*.
    """
    return sum(xi == yi for xi, yi in zip(x, y))

def matchSetStrength(match_set, target_set):
    """Compute the match strength of a set of strings on the target set of
    strings. The strength is the maximum of all match string on each target.
    """
    sum = 0.0
    for t in target_set:
        sum += max(matchStrength(m, t) for m in match_set)
    return sum / len(target_set),

def competitive_fitness(individuals, target_set):
    # Calculate OneMax scores for both individuals
    onemax_scores = [sum(ind) for ind in individuals]

    # Assign fitness based on competition
    fitness = 50 - onemax_scores[1]
    return fitness,

def GAeval(individual):
    fitness = 0
    
    for bit in individual:
        fitness += bit
    
    return fitness,

### **GA SPECIFICATIONS**

In [27]:
#Genetic Algorithm Specifications
toolbox.register("bit", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.bit, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("species", tools.initRepeat, list, toolbox.individual, SPECIES_SIZE)
toolbox.register("target_set", initTargetSet)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1./IND_SIZE) 
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("get_best", tools.selBest, k=1)

toolbox.register("eval", matchSetStrength)
toolbox.register("comp_eval", competitive_fitness)
toolbox.register("evaluate", GAeval)

## **Part 1: Cooperative GA on OneMax**

### **Random Pairing Strategy**

In [28]:
def coopRandomPair(extended=True, verbose=True):
    target_set = []
    species = []

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "species", "evals", "std", "min", "avg", "max"

    ngen = 200
    g = 0

    schematas = nicheSchematas(TARGET_TYPE, IND_SIZE)
    for i in range(TARGET_TYPE):
        size = int(TARGET_SIZE/TARGET_TYPE)
        target_set.extend(toolbox.target_set(schematas[i], size))
        species.append(toolbox.species())

    # Init with a random representative for each species
    representatives = [random.choice(s) for s in species]

    while g < ngen:
        # Initialize a container for the next generation representatives
        next_repr = [None] * len(species)
        for i, s in enumerate(species):
            # Vary the species individuals
            s = algorithms.varAnd(s, toolbox, 0.7, 0.01)

            # Get the representatives excluding the current species
            r = representatives[:i] + representatives[i+1:]
            for ind in s:
                ind.fitness.values = toolbox.eval([ind] + r, target_set)

            record = stats.compile(s)
            logbook.record(gen=g, species=i, evals=len(s), **record)

            if verbose: 
                print(logbook.stream)

            # Select the individuals
            species[i] = [random.choice(s) for i in range(len(s))]  # Random selection
            next_repr[i] = random.choice(s)   # Random Selection

            g += 1
        representatives = next_repr

    if extended:
        for r in representatives:
            print("".join(str(x) for x in r))

coopRandomPair()

gen	species	evals	std     	min  	avg    	max  
0  	0      	100  	0.394001	28.68	29.1129	30.85
1  	1      	100  	0.941142	25.3 	26.8522	29.63
2  	0      	100  	1.33098 	24.09	26.2098	30.62
3  	1      	100  	0.848559	24.95	26.7465	29.55
4  	0      	100  	1.14196 	24.87	26.7094	29.43
5  	1      	100  	1.10533 	23.69	25.9262	28.79
6  	0      	100  	1.38875 	23.37	26.2215	29.97
7  	1      	100  	0.863783	25.51	26.9929	29.35
8  	0      	100  	1.51262 	23.01	25.7373	29.53
9  	1      	100  	0.797227	26.71	27.7245	29.53
10 	0      	100  	0.805571	26.03	27.2946	29.95
11 	1      	100  	1.22309 	24.32	26.3508	29.42
12 	0      	100  	0.845432	26.29	27.6414	30.79
13 	1      	100  	0.495052	29.31	29.9305	31.45
14 	0      	100  	0.980133	24.62	26.5533	30.03
15 	1      	100  	1.20365 	24.67	26.7395	29.26
16 	0      	100  	0.4974  	26.77	27.5625	28.91
17 	1      	100  	0.739768	26.01	27.159 	29.45
18 	0      	100  	1.0861  	23.59	25.7429	28.73
19 	1      	100  	0.737689	26.15	27.3151	29.42
20 	0      	1

### **Best Pairing Strategy**

In [29]:
def coopBestPair(extended=True, verbose=True):
    target_set = []
    species = []

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "species", "evals", "std", "min", "avg", "max"

    ngen = 200
    g = 0

    schematas = nicheSchematas(TARGET_TYPE, IND_SIZE)
    for i in range(TARGET_TYPE):
        size = int(TARGET_SIZE/TARGET_TYPE)
        target_set.extend(toolbox.target_set(schematas[i], size))
        species.append(toolbox.species())

    # Init with a random representative for each species
    representatives = [random.choice(s) for s in species]

    while g < ngen:
        # Initialize a container for the next generation representatives
        next_repr = [None] * len(species)
        for i, s in enumerate(species):
            # Vary the species individuals
            s = algorithms.varAnd(s, toolbox, 0.7, 0.01)

            # Get the representatives excluding the current species
            r = representatives[:i] + representatives[i+1:]
            for ind in s:
                ind.fitness.values = toolbox.eval([ind] + r, target_set)

            record = stats.compile(s)
            logbook.record(gen=g, species=i, evals=len(s), **record)

            if verbose: 
                print(logbook.stream)

            # Select the individuals
            species[i] = toolbox.select(s, len(s))  # Tournament selection
            next_repr[i] = toolbox.get_best(s)[0]   # Best selection

            g += 1
        representatives = next_repr

    if extended:
        for r in representatives:
            print("".join(str(x) for x in r))

coopBestPair()

gen	species	evals	std     	min  	avg    	max  
0  	0      	100  	0.741163	26.57	27.7093	30.56
1  	1      	100  	1.1497  	24.21	26.1698	30.42
2  	0      	100  	0.567877	30.25	30.9493	34.28
3  	1      	100  	1.12777 	27.57	29.9727	33.32
4  	0      	100  	0.726711	30.39	31.4697	34.48
5  	1      	100  	0.8569  	29.65	31.899 	34.8 
6  	0      	100  	0.877143	31.18	32.507 	35.12
7  	1      	100  	0.675316	31.39	32.8406	35.01
8  	0      	100  	1.07501 	31.66	33.3492	35.2 
9  	1      	100  	0.747007	31.9 	33.4854	35.12
10 	0      	100  	0.799577	32.3 	34.3398	35.4 
11 	1      	100  	0.669627	32.64	34.176 	35.34
12 	0      	100  	0.323408	33.2 	35.0812	35.74
13 	1      	100  	0.405449	33.48	34.9375	35.46
14 	0      	100  	0.169152	34.42	35.1343	35.6 
15 	1      	100  	0.275018	34.18	35.4236	35.7 
16 	0      	100  	0.285317	34.18	35.3094	36.4 
17 	1      	100  	0.100558	35.04	35.5404	36.04
18 	0      	100  	0.326752	35.13	35.9449	36.82
19 	1      	100  	0.128425	36.26	36.3852	36.9 
20 	0      	1

### **Analysis:**

 The analysis of pairing strategies in the context of fitness values reveals compelling insights. Notably, the best pairing strategy consistently outperforms the random pairing approach, showcasing higher average fitness values. Moreover, the standard deviation associated with the best pairing strategy is notably lower, indicative of more consistent and stable results. In contrast, the random pairing strategy lacks convergence in fitness values, suggesting a lack of systematic improvement over iterations. This comparison underscores the effectiveness of employing a well-defined pairing strategy for achieving superior and more reliable outcomes in the optimization process.

## **Part 2: Competitive GA on OneMax**

### **Random Pairing Strategy**

In [30]:
def competRandomPair(extended=True, verbose=True):
    target_set = []
    species = []

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "species", "evals", "std", "min", "avg", "max"

    ngen = 200
    g = 0

    schematas = nicheSchematas(TARGET_TYPE, IND_SIZE)
    for i in range(TARGET_TYPE):
        size = int(TARGET_SIZE/TARGET_TYPE)
        target_set.extend(toolbox.target_set(schematas[i], size))
        species.append(toolbox.species())

    # Init with a random representative for each species
    representatives = [random.choice(s) for s in species]

    while g < ngen:
        # Initialize a container for the next generation representatives
        next_repr = [None] * len(species)
        for i, s in enumerate(species):
            # Vary the species individuals
            s = algorithms.varAnd(s, toolbox, 0.7, 0.01)

            # Get the representatives excluding the current species
            r = representatives[:i] + representatives[i+1:]
            for ind in s:
                ind.fitness.values = toolbox.comp_eval([ind] + r, target_set)

            record = stats.compile(s)
            logbook.record(gen=g, species=i, evals=len(s), **record)

            if verbose: 
                print(logbook.stream)

            # Select the individuals
            species[i] = [random.choice(s) for i in range(len(s))]  # Random selection
            next_repr[i] = random.choice(s)   # Random Selection

            g += 1
        representatives = next_repr

    if extended:
        for r in representatives:
            print("".join(str(x) for x in r))

competRandomPair()

gen	species	evals	std	min	avg	max
0  	0      	100  	0  	24 	24 	24 
1  	1      	100  	0  	27 	27 	27 
2  	0      	100  	0  	26 	26 	26 
3  	1      	100  	0  	33 	33 	33 
4  	0      	100  	0  	26 	26 	26 
5  	1      	100  	0  	30 	30 	30 
6  	0      	100  	0  	24 	24 	24 
7  	1      	100  	0  	25 	25 	25 
8  	0      	100  	0  	22 	22 	22 
9  	1      	100  	0  	29 	29 	29 
10 	0      	100  	0  	21 	21 	21 
11 	1      	100  	0  	20 	20 	20 
12 	0      	100  	0  	24 	24 	24 
13 	1      	100  	0  	24 	24 	24 
14 	0      	100  	0  	27 	27 	27 
15 	1      	100  	0  	25 	25 	25 
16 	0      	100  	0  	24 	24 	24 
17 	1      	100  	0  	21 	21 	21 
18 	0      	100  	0  	22 	22 	22 
19 	1      	100  	0  	27 	27 	27 
20 	0      	100  	0  	25 	25 	25 
21 	1      	100  	0  	27 	27 	27 
22 	0      	100  	0  	20 	20 	20 
23 	1      	100  	0  	19 	19 	19 
24 	0      	100  	0  	23 	23 	23 
25 	1      	100  	0  	29 	29 	29 
26 	0      	100  	0  	25 	25 	25 
27 	1      	100  	0  	25 	25 	25 
28 	0      	10

### **Best Pairing Strategy**

In [31]:
def competBestPair(extended=True, verbose=True):
    target_set = []
    species = []

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "species", "evals", "std", "min", "avg", "max"

    ngen = 200
    g = 0

    schematas = nicheSchematas(TARGET_TYPE, IND_SIZE)
    for i in range(TARGET_TYPE):
        size = int(TARGET_SIZE/TARGET_TYPE)
        target_set.extend(toolbox.target_set(schematas[i], size))
        species.append(toolbox.species())

    # Init with a random representative for each species
    representatives = [random.choice(s) for s in species]

    while g < ngen:
        # Initialize a container for the next generation representatives
        next_repr = [None] * len(species)
        for i, s in enumerate(species):
            # Vary the species individuals
            s = algorithms.varAnd(s, toolbox, 0.7, 0.01)

            # Get the representatives excluding the current species
            r = representatives[:i] + representatives[i+1:]
            for ind in s:
                ind.fitness.values = toolbox.comp_eval([ind] + r, target_set)

            record = stats.compile(s)
            logbook.record(gen=g, species=i, evals=len(s), **record)

            if verbose: 
                print(logbook.stream)

            # Select the individuals
            species[i] = toolbox.select(s, len(s))  # Tournament selection
            next_repr[i] = toolbox.get_best(s)[0]   # Best selection

            g += 1
        representatives = next_repr

    if extended:
        for r in representatives:
            print("".join(str(x) for x in r))

competBestPair()

gen	species	evals	std	min	avg	max
0  	0      	100  	0  	22 	22 	22 
1  	1      	100  	0  	26 	26 	26 
2  	0      	100  	0  	24 	24 	24 
3  	1      	100  	0  	25 	25 	25 
4  	0      	100  	0  	25 	25 	25 
5  	1      	100  	0  	25 	25 	25 
6  	0      	100  	0  	21 	21 	21 
7  	1      	100  	0  	34 	34 	34 
8  	0      	100  	0  	21 	21 	21 
9  	1      	100  	0  	21 	21 	21 
10 	0      	100  	0  	23 	23 	23 
11 	1      	100  	0  	33 	33 	33 
12 	0      	100  	0  	24 	24 	24 
13 	1      	100  	0  	22 	22 	22 
14 	0      	100  	0  	22 	22 	22 
15 	1      	100  	0  	26 	26 	26 
16 	0      	100  	0  	25 	25 	25 
17 	1      	100  	0  	21 	21 	21 
18 	0      	100  	0  	21 	21 	21 
19 	1      	100  	0  	17 	17 	17 
20 	0      	100  	0  	31 	31 	31 
21 	1      	100  	0  	23 	23 	23 
22 	0      	100  	0  	25 	25 	25 
23 	1      	100  	0  	25 	25 	25 
24 	0      	100  	0  	28 	28 	28 
25 	1      	100  	0  	25 	25 	25 
26 	0      	100  	0  	25 	25 	25 
27 	1      	100  	0  	22 	22 	22 
28 	0      	10

### **ANALYSIS:**

Surprisingly, the random pairing strategy exhibited consistent fitness values across generations, indicating no improvement or exploration, while the best pairing strategy, designed to select the fittest individuals for reproduction, showed some variability in fitness values, implying a potential for exploration and the discovery of better solutions. Despite the expected advantages of the best pairing approach, the competitive nature of the algorithm might have influenced the outcomes, leading to the unexpected result of random pairing performing better. Further investigation into the specific dynamics and interactions within the coevolutionary algorithm, along with potential adjustments to parameters could be done to further teh results for best pairing strategy.

## **Part 3: Comparison of Approaches and Coevolutionary Application**


### **GA Implementation of OneMax**

In [32]:
def GAOneMax():

    pop = toolbox.population(n=SPECIES_SIZE)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.01, ngen=200, verbose=True)
    best = [str(i) for i in toolbox.get_best(pop)[0]]
    print("".join(best))
    print(toolbox.evaluate(toolbox.get_best(pop)[0])[0])

GAOneMax()

gen	nevals
0  	100   
1  	81    
2  	82    
3  	67    
4  	63    
5  	62    
6  	70    
7  	76    
8  	78    
9  	68    
10 	70    
11 	78    
12 	66    
13 	68    
14 	74    
15 	64    
16 	71    
17 	66    
18 	64    
19 	73    
20 	73    
21 	78    
22 	68    
23 	64    
24 	70    
25 	59    
26 	68    
27 	74    
28 	78    
29 	78    
30 	72    
31 	57    
32 	64    
33 	68    
34 	68    
35 	72    
36 	62    
37 	61    
38 	64    
39 	72    
40 	66    
41 	78    
42 	76    
43 	48    
44 	64    
45 	73    
46 	66    
47 	72    
48 	74    
49 	68    
50 	62    
51 	88    
52 	76    
53 	65    
54 	83    
55 	80    
56 	68    
57 	74    
58 	62    
59 	74    
60 	76    
61 	78    
62 	61    
63 	78    
64 	75    
65 	71    
66 	70    
67 	70    
68 	68    
69 	77    
70 	74    
71 	67    
72 	70    
73 	75    
74 	65    
75 	79    
76 	67    
77 	62    
78 	63    
79 	78    
80 	64    
81 	68    
82 	67    
83 	80    
84 	78    
85 	68    
86 	83    
87 	84    
88 	72    
89 	70    

### **COMPARISON AND DISCUSSION:**

Genetic Algorithms (GAs) work really well for OneMax problems as they tend to be faster, more efficient, and consistently reach higher fitness values, usually close to the maximum of 50. This is due to the straight forward approach of GA combined with the simplicity of the problem at hand. Among the coevolutionary approaches, the Cooperative approach is better than Competitive when it comes to the OneMax problem, as it tends to achieve higher fitness values. Specifically, when using cooperative appriach on a OneMax problem, it's best to use the best pairing strategy for better results. On the other hand, for competitive approach, a random pairing strategy works better, making the algorithm more flexible and improving its chances of finding the best solution.

Coevolutionary approaches prove advantageous in addressing complex and dynamic problems, such as evolving strategies for game playing, where intricate interactions between entities are crucial for success. These approaches, however, may not be beneficial for simpler problem domains or scenarios where well-defined relationships exist between components. The potential drawbacks include the introduction of unnecessary complexity and computational overhead in situations where a single evolving population can efficiently find solutions. Sensitivity to parameter choices, the risk of stagnation, and the computational cost associated with evaluating fitness in coevolutionary settings further emphasize the need for careful consideration of the problem's nature and complexity before opting for a coevolutionary approach.

## **Part 4: Real World Problem** 

One real world application of co-evolutionary programming is simulating strategic behavior in markets. Prize, in his paper titled "Using Co-Evolutionary Programming to Simulate Strategic Behaviour in Markets" showed how co-evolutionary programming could be utilized in this manner.

This paper explores the application of a genetic algorithm (GA) in modeling standard industrial organization games, including Bertrand and Cournot competition, a vertical chain of monopolies, and a simple electricity pool model. The study aims to demonstrate the effectiveness of GAs as modeling tools in these common settings, suggesting their potential role in applied work requiring detailed market simulation. The advantages of using GAs over traditional scenario analysis for market simulation are discussed, emphasizing the potential of evolutionary programming (EP) in making applied conclusions more robust and less arbitrary. The conclusion highlights the success of EP in searching for equilibria in simple games but acknowledges the need for more sophisticated structures, such as genetic programming, as choice problems become more complex. Genetic programming is considered promising for evolving computer programs to represent firm behavior and decision-making processes, offering a clear analogue to real-world decision-making aids used by firms and regulators.

### **SOURCE:**
Price, T. C. (1997). *Using Co-Evolutionary Programming to Simulate Strategic Behaviour in Markets. Journal of Evolutionary Economics, 7*(3), 219–254. https://doi.org/10.1007/s001910050042